In [ ]:
import cv2
import matplotlib.pyplot as plt
from matplotlib import animation

from dataset import VOT14Reader
from trackers import ColorMSTracker

In [ ]:
dataset = VOT14Reader(dataset_path="./vot14")

sample = dataset[13] # 13 for ball
polygon = sample[1][0]
frame = sample[0][0]

roi = frame[polygon[1]:polygon[1] + polygon[3],
            polygon[0]:polygon[0] + polygon[2]]

term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)
tracker = ColorMSTracker(tracked_window=roi, term_crit=term_crit)

In [ ]:
fig, ax = plt.subplots()
images = []
for frame in sample[0][1:]:
    polygon = tracker.track(frame, polygon)
    x, y, w, h = polygon
    final_image = cv2.rectangle(frame, (x, y), (x+w, y+h), 255, 3)
    images.append([ax.imshow(final_image, animated=True)])

In [13]:
ani = animation.ArtistAnimation(fig, images, interval=50, blit=True, repeat_delay=1000)

In [14]:
from IPython.display import HTML
HTML(ani.to_html5_video())
